In [77]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [78]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-03-07 12:51:21--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:221, 2606:4700:20::ac43:4695, 2606:4700:20::681a:321, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.4’

train-data.tsv.4    100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-03-07 12:51:22 (14.1 MB/s) - ‘train-data.tsv.4’ saved [358233/358233]

--2024-03-07 12:51:22--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:221, 2606:4700:20::ac43:4695, 2606:4700:20::681a:321, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/

In [79]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
test_data = pd.read_csv(test_file_path, sep='\t', header=None)

train_data.columns = ['class', 'sms']
test_data.columns = ['class', 'sms']

train_labels = train_data.pop('class')
test_labels = test_data.pop('class')

class_map = {'ham':0, 'spam':1}
train_labels = train_labels.map(class_map)
test_labels = test_labels.map(class_map)

train_data = np.array(train_data['sms'])
test_data = np.array(test_data['sms'])

test_labels = np.array(test_labels)
train_labels = np.array(train_labels)


In [80]:
vectorize_layer = tf.keras.layers.TextVectorization()
vectorize_layer.adapt(train_data)
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())


In [81]:
model = keras.Sequential([
    vectorize_layer,
    keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ['acc'])

In [82]:
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.25)

Epoch 1/10
98/98 [==============================] - 3s 16ms/step - loss: 0.3179 - acc: 0.8918 - val_loss: 0.1226 - val_acc: 0.9780
Epoch 2/10
98/98 [==============================] - 1s 12ms/step - loss: 0.0955 - acc: 0.9799 - val_loss: 0.0854 - val_acc: 0.9809
Epoch 3/10
98/98 [==============================] - 1s 11ms/step - loss: 0.0571 - acc: 0.9888 - val_loss: 0.0710 - val_acc: 0.9847
Epoch 4/10
98/98 [==============================] - 1s 11ms/step - loss: 0.0408 - acc: 0.9917 - val_loss: 0.0739 - val_acc: 0.9866
Epoch 5/10
98/98 [==============================] - 1s 11ms/step - loss: 0.0314 - acc: 0.9946 - val_loss: 0.0668 - val_acc: 0.9876
Epoch 6/10
98/98 [==============================] - 1s 11ms/step - loss: 0.0315 - acc: 0.9930 - val_loss: 0.1245 - val_acc: 0.9732
Epoch 7/10
98/98 [==============================] - 1s 11ms/step - loss: 0.0231 - acc: 0.9962 - val_loss: 0.2308 - val_acc: 0.9560
Epoch 8/10
98/98 [==============================] - 1s 11ms/step - loss: 0.0265 - a

In [83]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict(np.array([pred_text]))
  prediction = [pred[0][0], 'ham'] if pred[0][0] < 0.5 else [pred[0][0], 'spam']
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 282ms/step
[0.13440558, 'ham']


In [84]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 13ms/step
You passed the challenge. Great job!
